# Indigenous lands on which observatories are located

In [1]:
import pandas as pd
import folium
import numpy as np
import math
from astropy.coordinates import EarthLocation as EL
import astropy.units as u
import requests
from shapely.geometry import Polygon

In [2]:
sites = EL.get_site_names()
sites_csv = pd.read_csv('Observatory Locations - Sheet1.csv')

In [3]:
folium_map = folium.Map(location=[43.6532, -79.3832],
                        zoom_start=3,
                        tiles="OpenStreetMap")

In [4]:
infos_allsites = []
for _, site in sites_csv.iterrows():
    site_name = site[0]
    loc =EL.from_geodetic(lat=site[1]*u.deg, lon=site[2]*u.deg).geodetic
    locstr = '{:.6f},{:.6f}'.format(loc[1].to(u.deg).value, loc[0].to(u.deg).value)

    response = requests.get("https://native-land.ca/api/index.php?maps=territories&position={:s}".format(locstr))
    infos = response.json()
    infos_allsites.extend(infos)
    namelist = [info['properties']['Name'] for info in infos]

    namestr = ''
    c = 0
    for name in namelist:
        if c==(len(namelist)-1):
            namestr=namestr+"{:s}".format(name)
        else:
            namestr=namestr+"{:s},\n".format(name)
        c+=1
    popup_text = "Observatory Name: {}<br> Indigenous Land: {}"
    popup_text = popup_text.format(site_name,
                          namestr)
    popup = folium.Popup(popup_text, max_width=200)

    folium.Marker(location = [loc[1].to(u.deg).value, loc[0].to(u.deg).value], popup=popup).add_to(folium_map)


coords = [info['geometry']['coordinates'][0] for info in infos_allsites]
polygons = [Polygon(l) for l in coords]
a = [p.area for p in polygons]
order = np.argsort(a)
infos_allsites = np.asarray(infos_allsites)
m = 0
for info in infos_allsites[order][::-1]:
    territory_name = info['properties']['Name']
    territory_color = info['properties']['color']


    namelist.append(territory_name)
    cp = folium.Choropleth(info['geometry'],name=territory_name, overlay=True, fill_color=territory_color,
            fill_opacity=.1, line_color=territory_color, line_opacity=1, highlight=True).add_to(folium_map)

    folium.Tooltip(territory_name).add_to(cp.geojson)
    m+=1

In [5]:
folium_map